# Activar infraestructura de GPUs

In [ ]:
#Como nuestros datos tienen representaciones matriciales y tensoriales, podemos usar hardware especializado
#Las GPUs está especializadas en procesar estos tipos de estructuras de datos
#En la opcion "Entorno de ejecucion", seleccionamos "Cambiar tipo de entorno de ejecucion"
#En la casilla de acelerador de Hardware, seleccionamos "GPU"
#Notemos como se nos está re-asignando una nueva infraestructura

# Librerías

In [2]:
#Importamos las librerías estándar
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

# Conexión al repositorio de datos

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Lectura del dataset MNIST

In [4]:
#Para estar evitando el convertir las imágenes a tensores, usaremos este dataset de pruebas
#Vamos a importarlo
#En total tiene 70000 imágenes
from keras.datasets import mnist

#Importamos los datos de entrenamiento y testing
#La extensión NPZ significa "NumPy Zip", es un array de numpy comprimido
#La librería colocará las imágenes en el disco "/tmp"
#Luego con la función "load_data" dividimos los datos en entrenamiento y validación
#Esta función es propia de "Mnist"
(x_train, y_train), (x_test, y_test) = mnist.load_data('/tmp/mnist.npz')

# Procesamiento de features

In [5]:
#Tenemos 60 mil imágenes de 28x28 píxeles que representan números escritos a mano
x_train.shape

(60000, 28, 28)

In [6]:
#Tenemos que representar este dataset como un tensor de orden 4 del tipo: (dataset, filas, columnas, canales)
x_tensor_train = x_train.reshape(-1, 28, 28, 1)
x_tensor_train.shape

(60000, 28, 28, 1)

In [7]:
#Escalamos los features de entrenamiento
x_tensor_train_scale = x_tensor_train.astype('float32') / 255.0
x_tensor_train_scale.shape

(60000, 28, 28, 1)

In [8]:
#Tenemos 10 mil imágenes de 28x28 píxeles que representan números escritos a mano para los datos de validacion
x_test.shape

(10000, 28, 28)

In [9]:
#Hacemos lo mismo con los datos de validación
x_tensor_test = x_test.reshape(-1, 28, 28, 1)
x_tensor_test.shape

(10000, 28, 28, 1)

In [10]:
#Escalamos los features de validacion
x_tensor_test_scale = x_tensor_test.astype('float32') / 255.0
x_tensor_test_scale.shape

(10000, 28, 28, 1)

# Procesamiento de labels

In [11]:
#Cada imagen tiene un número asociado que indica qué significa el número escrito a mano
#Por ejemplo, la primera imágen corresponde al número 5, la segunda al número 0
y_train

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [12]:
#Importamos la librería que categoriza
from keras.utils.np_utils import to_categorical

In [13]:
#Categorizamos los labels de entrenamiento
y_train_categorico = to_categorical(y_train)
y_train_categorico.shape

(60000, 10)

In [14]:
#Por ejemplo, vemos que en la primera imagen, su label asociado está activado en el índice 5, entonces la imagen es un número 5
y_train_categorico[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

In [15]:
#Hacemos lo mismo con los labels de validación
y_test_categorico = to_categorical(y_test)
y_test_categorico.shape

(10000, 10)

# Construcción de Red Convolucional para encontrar patrones visuales

In [16]:
#Importamos las librerías para definir modelos de Deep Learning
from keras.models import Sequential

In [17]:
#Importamos los objetos que nos permiten definir una capa convolucional para procesar imágenes en 2D
from keras.layers import Conv2D
from keras.layers import MaxPool2D, AvgPool2D

In [18]:
#Instaciamos un modelo vacío
model = Sequential()

In [19]:
#Agregamos una capa, de 16 filtros, cada filtro de 4x4
#El input es una imagen de 28x28 pixeles de 1 canal de color
model.add(Conv2D(16, (4, 4), input_shape=(28, 28, 1)))

In [20]:
model.add(MaxPool2D(4, (4, 4)))

# Vectorizacion de los patrones visuales

In [21]:
#Ahora los tensores patrones encontrados para cada imagen, los tenemos que convertir a vectores de features
#Importamos la función que vectoriza los tensores
from keras.layers import Flatten

In [22]:
#Vectorizamos los patrones
model.add(Flatten())

# Red neuronal para clasificar los patrones

In [23]:
#Importamos el objeto para definir capas de redes neuronales
from keras.layers import Dense

In [24]:
#Agregamos capas de nuestra red neuronal, 128 neuronas
model.add(Dense(128, activation='relu'))

In [25]:
#Como estamos clasificando, agregamos una capa de salida de 10 neuronas con softmax
model.add(Dense(10, activation='softmax'))

# Compilación del modelo de Deep Learning (Red Convolucional + Red Neuronal)

In [26]:
#Compilamos
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenamiento del modelo

In [27]:
#AL ejecutarlo, notamos que se entrena muy lento
#Esto se debe a la gran cantidad de información que se está procesando, matricialmente cada imagen es de 28x28, 
model.fit(x_tensor_train_scale, y_train_categorico, batch_size=128, epochs=2, verbose=1)

Epoch 1/2
469/469 [==============================] - 33s 6ms/step - loss: 0.3145 - accuracy: 0.9100
Epoch 2/2
469/469 [==============================] - 3s 6ms/step - loss: 0.0928 - accuracy: 0.9718


# Uso del modelo en un archivo

In [28]:
#Importamos la libreria de imagenes
from keras.preprocessing import image

In [44]:
#Leemos la imagen
imagen = image.load_img(
  '/content/drive/MyDrive/Data/PruebaDe7.PNG', #imagen_5.png', 
  grayscale=False,
  color_mode="grayscale",
  target_size = (28, 28),
  interpolation="nearest"
)
imagen

In [45]:
#La convertimos en un tensor
tensor = np.array([
  image.img_to_array(imagen)
])

#Vemos el tensor
tensor.shape

(1, 28, 28, 1)

In [46]:
#Usamos el modelo para predecir
predicciones_labels = model.predict(tensor)
predicciones_labels

array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]], dtype=float32)

In [47]:
#Obtenemos el índice del array de predicciones para saber a qué categoría pertenece
predicciones = predicciones_labels.argmax()
predicciones

7

# Uso del modelo en una carpeta de imágenes

In [48]:
#Importamos la librería para listar archivos
import glob

#Implementamos una función que itera para obtener el tensor de todas las imágenes en una ruta
def leer_carpeta_de_imagenes_como_tensor(ruta):
  tensores = []

  #Obtenemos las rutas de las imágenes
  rutas_imagenes = sorted(glob.glob(ruta+'/*'))

  index = 0
  for ruta in rutas_imagenes:
    #Leemos la imagen
    imagen = image.load_img(
      ruta, 
      grayscale=False,
      color_mode="grayscale",
      target_size = (28, 28),
      interpolation="nearest"
    )

    #La tensorizamos
    tensor = np.array([image.img_to_array(imagen)])

    #La agregamos a la lista de tensores
    if index == 0:
      tensores = tensor
    else:
      tensores = np.concatenate((tensores, tensor))

    index = index + 1

  #La retornamos
  return tensores

In [49]:
#Leemos las imagenes como tensores
tensores = leer_carpeta_de_imagenes_como_tensor('/content/drive/MyDrive/Data/MNIST')
tensores.shape

(6, 28, 28, 1)

In [50]:
#Usamos el modelo para predecir
predicciones_labels = model.predict(tensores)
predicciones_labels

array([[0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 1.000000e+00,
        0.000000e+00, 0.000000e+00],
       [0.000000e+00, 0.000000e+00, 0.000000e+00, 1.000000e+00,
        0.000000e+00, 2.927459e-30, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       [1.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       [0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        1.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       [0.000000e+00, 1.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       [0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 1.000000e+00]], dtype=flo

In [51]:
#Obtenemos el índice del array de predicciones para saber a qué categoría pertenece
predicciones = predicciones_labels.argmax(axis = 1)
predicciones

array([7, 3, 0, 4, 1, 9])